In [13]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import model_from_json

In [15]:
#Prakhar n Sankalp
right = np.loadtxt("right_data.txt")
right_C_1=np.loadtxt("right_data_C_1.txt")
right_C_3=np.loadtxt("right_data_C.txt")
left = np.loadtxt("left_data.txt")
left_C_1=np.loadtxt("left_data_C_1.txt")
left_C_3=np.loadtxt("left_data_C.txt")



right_reshaped = np.reshape(right, (right.shape[0], right.shape[1] // 2, 2))
left_reshaped = np.reshape(left, (left.shape[0], left.shape[1] // 2, 2))
right_C_1_reshaped = np.reshape(right_C_1, (right.shape[0], right.shape[1] // 2, 2))
left_C_1_reshaped = np.reshape(left_C_1, (left.shape[0], left.shape[1] // 2, 2))
right_C_3_reshaped = np.reshape(right_C_3, (right.shape[0], right.shape[1] // 2, 2))
left_C_3_reshaped = np.reshape(left_C_3, (left.shape[0], left.shape[1] // 2, 2))



right_label = np.zeros((3*right.shape[0], 2))
right_label[:,0] = 1
left_label = np.zeros((3*left.shape[0], 2))
left_label[:, 1] = 1





right_train=right_reshaped[:int(0.8*2352)]
right_C_1_train=right_C_1_reshaped[:int(0.8*2352)]
right_C_3_train=right_C_3_reshaped[:int(0.8*2352)]

right_test=right_reshaped[int(0.8*2352):]
right_C_1_test=right_C_1_reshaped[int(0.8*2352):]
right_C_3_test=right_C_3_reshaped[int(0.8*2352):]



right_label_train=right_label[:3*int(0.8*2352)]
right_label_test=right_label[3*int(0.8*2352):]





left_train=left_reshaped[:int(0.8*2396)]
left_C_1_train=left_C_1_reshaped[:int(0.8*2396)]
left_C_3_train=left_C_3_reshaped[:int(0.8*2396)]

left_test=left_reshaped[int(0.8*2396):]
left_C_1_test=left_C_1_reshaped[int(0.8*2396):]
left_C_3_test=left_C_3_reshaped[int(0.8*2396):]



left_label_train=left_label[:3*int(0.8*2396)]
left_label_test=left_label[3*int(0.8*2396):]


In [18]:
data_train=np.concatenate((right_train, left_train), axis = 0)
data_train_C_1= np.concatenate((right_C_1_train, left_C_1_train), axis = 0)
data_train_C_3=np.concatenate((right_C_3_train, left_C_3_train), axis = 0)
data_final_train=np.vstack((data_train,data_train_C_1,data_train_C_3))

data_test=np.concatenate((right_test, left_test), axis = 0)
data_test_C_1= np.concatenate((right_C_1_test, left_C_1_test), axis = 0)
data_test_C_3=np.concatenate((right_C_3_test, left_C_3_test), axis = 0)
data_final_test=np.vstack((data_test,data_test_C_1,data_test_C_3))

label_train = np.concatenate((right_label_train, left_label_train), axis = 0)
label_test = np.concatenate((right_label_test, left_label_test), axis = 0)

print(data_final_train.shape)
print(label_train.shape)

print(data_final_test.shape)

(11391, 640, 2)
(11391, 2)
(2853, 640, 2)


In [19]:
randomize_1= np.arange(0, data_final_train.shape[0], 1)
randomize_1 = np.random.permutation(randomize_1)

randomize_2= np.arange(0, data_final_test.shape[0], 1)
randomize_2 = np.random.permutation(randomize_2)

data_final_train = data_final_train[randomize_1]
label_train = label_train[randomize_1]
data_final_test = data_final_test[randomize_2]
label_test = label_test[randomize_2]


In [20]:
normalizer = keras.layers.experimental.preprocessing.Normalization(mean=0., variance=1.)
normalizer.adapt(data_final_train)

In [21]:
inputs = keras.Input(shape=(640, 2, 1))

In [22]:
from tensorflow.keras import layers

x = normalizer(inputs)

x = layers.Conv2D(filters=25, kernel_size=[11,1], strides=[1,1], padding="valid")(x)
x = layers.LeakyReLU(alpha=0.01)(x)
x = layers.SpatialDropout2D(rate=0.5)(x)

x = layers.Conv2D(filters=25, kernel_size=[1,2], strides=[1,1], padding="valid")(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU(alpha=0.01)(x)

x = layers.MaxPooling2D(pool_size=[3,1], strides = [3,1], padding="valid")(x)

x = layers.Conv2D(filters=50, kernel_size=[11,1], strides=[1,1], padding="valid")(x)
x = layers.LeakyReLU(alpha=0.01)(x)
x = layers.SpatialDropout2D(rate=0.5)(x)

x = layers.MaxPooling2D(pool_size=[3,1], strides = [3,1], padding="valid")(x)

x = layers.Conv2D(filters=100, kernel_size=[11,1], strides=[1,1], padding="valid")(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU(alpha=0.01)(x)
x = layers.SpatialDropout2D(rate=0.5)(x)

x = layers.MaxPooling2D(pool_size=[3,1], strides = [3,1], padding="valid")(x)

x = layers.Conv2D(filters=200, kernel_size=[11,1], strides=[1,1], padding="valid")(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU(alpha=0.01)(x)

x = layers.MaxPooling2D(pool_size=[2,1], strides = [2,1], padding="valid")(x)

x = layers.Flatten()(x)

num_classes = 2
outputs = layers.Dense(num_classes, activation="softmax")(x)

In [23]:
model = keras.Model(inputs=inputs, outputs=outputs)

In [24]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 640, 2, 1)]       0         
_________________________________________________________________
normalization_1 (Normalizati (None, 640, 2, 2)         5         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 630, 2, 25)        575       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 630, 2, 25)        0         
_________________________________________________________________
spatial_dropout2d_3 (Spatial (None, 630, 2, 25)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 630, 1, 25)        1275      
_________________________________________________________________
batch_normalization_3 (Batch (None, 630, 1, 25)        100 

In [25]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5),
              loss=keras.losses.BinaryCrossentropy(), metrics=keras.metrics.BinaryAccuracy())

In [26]:
for i in range(8):
    history = model.fit(data_final_train,label_train,
              batch_size=5, epochs=25)
    model.evaluate(data_final_test,label_test)

Epoch 1/25
2279/2279 [==============================] - 29s 12ms/step - loss: 0.6967 - binary_accuracy: 0.6086
Epoch 2/25
2279/2279 [==============================] - 28s 12ms/step - loss: 0.6365 - binary_accuracy: 0.6533
Epoch 3/25
2279/2279 [==============================] - 28s 12ms/step - loss: 0.6041 - binary_accuracy: 0.6830
Epoch 4/25
2279/2279 [==============================] - 28s 12ms/step - loss: 0.5979 - binary_accuracy: 0.6882
Epoch 5/25
2279/2279 [==============================] - 28s 12ms/step - loss: 0.5870 - binary_accuracy: 0.6979
Epoch 6/25
2279/2279 [==============================] - 29s 13ms/step - loss: 0.5706 - binary_accuracy: 0.7096
Epoch 7/25
2279/2279 [==============================] - 28s 12ms/step - loss: 0.5683 - binary_accuracy: 0.7105
Epoch 8/25
2279/2279 [==============================] - 31s 14ms/step - loss: 0.5681 - binary_accuracy: 0.7143
Epoch 9/25
2279/2279 [==============================] - 29s 13ms/step - loss: 0.5610 - binary_accuracy: 0.7137
E

2279/2279 [==============================] - 44s 19ms/step - loss: 0.4316 - binary_accuracy: 0.8008
Epoch 22/25
2279/2279 [==============================] - 46s 20ms/step - loss: 0.4319 - binary_accuracy: 0.8033
Epoch 23/25
2279/2279 [==============================] - 44s 19ms/step - loss: 0.4320 - binary_accuracy: 0.8025
Epoch 24/25
2279/2279 [==============================] - 45s 20ms/step - loss: 0.4284 - binary_accuracy: 0.8054
Epoch 25/25
90/90 [==============================] - 3s 28ms/step - loss: 0.4448 - binary_accuracy: 0.8184
Epoch 1/25
2279/2279 [==============================] - 48s 21ms/step - loss: 0.4296 - binary_accuracy: 0.8049
Epoch 2/25
2279/2279 [==============================] - 54s 24ms/step - loss: 0.4273 - binary_accuracy: 0.8073 5s - loss:
Epoch 3/25
2279/2279 [==============================] - 50s 22ms/step - loss: 0.4309 - binary_accuracy: 0.8033
Epoch 4/25
2279/2279 [==============================] - 48s 21ms/step - loss: 0.4258 - binary_accuracy: 0.8081
Ep

KeyboardInterrupt: 

In [27]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk
